<a href="https://colab.research.google.com/github/Abhinav-1604/gdsc-embedded-induction/blob/main/KWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install librosa numpy soundfile tqdm


In [2]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
SAMPLE_RATE = 16000
DURATION = 1.0
SAMPLES = int(SAMPLE_RATE*DURATION)
N_MFCC = 13
FRAME_RATE = int(0.03*SAMPLE_RATE)
HOP_LENGTH = int(0.02*SAMPLE_RATE)
def load_audio(path):
 audio, sr = librosa.load(path, sr=SAMPLE_RATE)
 if len(audio)<SAMPLES:
  audio = np.pad(audio, (0, SAMPLES - len(audio)))
 else:
  audio = audio[:SAMPLES]
 return audio



In [4]:
from numpy._core.defchararray import center
def extract_mfcc(audio):
    mfcc = librosa.feature.mfcc(
        y=audio,
        sr=SAMPLE_RATE,
        n_mfcc=N_MFCC,
        n_fft=FRAME_RATE,
        hop_length=HOP_LENGTH,
        center=False
    )
    return mfcc.T


In [5]:
DATASET_PATH = "/content/drive/MyDrive/dataset"
CLASSES = ["on", "off", "unknown"]
label_map = { label: idx for idx, label in enumerate(CLASSES) }
X = []
y = []
for label in CLASSES:
    folder = os.path.join(DATASET_PATH, label)
    files = os.listdir(folder)
    print(f"Processing: {label} ({len(files)} files)")
    for f in tqdm(files):
        path = os.path.join(folder, f)

        audio = load_audio(path)
        mfcc = extract_mfcc(audio)

        X.append(mfcc)
        y.append(label_map[label])

X = np.array(X)
y = np.array(y)
X = X[..., np.newaxis]

print(X.shape)
print(y.shape)


Processing: on (2367 files)


100%|██████████| 2367/2367 [01:14<00:00, 31.65it/s] 


Processing: off (2357 files)


100%|██████████| 2357/2357 [01:00<00:00, 39.05it/s] 


Processing: unknown (1519 files)


100%|██████████| 1519/1519 [00:34<00:00, 44.28it/s] 

(6243, 49, 13, 1)
(6243,)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, shuffle=True
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, shuffle=True
)

print("Train:", X_train.shape)
print("Validation:", X_val.shape)
print("Test:", X_test.shape)


Train: (4370, 49, 13, 1)
Validation: (936, 49, 13, 1)
Test: (937, 49, 13, 1)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Dense, GlobalAveragePooling2D, BatchNormalization, ReLU, Reshape
from tensorflow.keras.optimizers import Adam

def build_dscnn(input_shape=(49, 13, 1), num_classes=3):
    model = Sequential()

    # Standard Conv layer (feature extractor)
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1),
                     padding='same', use_bias=False, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Depthwise Separable Convolution blocks
    for filters in [64, 128, 128]:
        # Depthwise
        model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1),
                                  padding='same', use_bias=False))
        model.add(BatchNormalization())
        model.add(ReLU())

        # Pointwise
        model.add(Conv2D(filters=filters, kernel_size=(1, 1),
                         strides=(1, 1), padding='same', use_bias=False))
        model.add(BatchNormalization())
        model.add(ReLU())

    # Global Average Pooling → removes time/freq dimensions
    model.add(GlobalAveragePooling2D())

    # Final classifier
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_dscnn()
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 49, 13, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 49, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 49, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 49, 13, 64)     │           576 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 49, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 49, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 49, 13, 64)     │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 49, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 49, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 49, 13, 64)     │           576 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 49, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 49, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 49, 13, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 49, 13, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 49, 13, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (None, 49, 13, 128)    │         1,152 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 49, 13, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 49, 13, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 49, 13, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 49, 13, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 34,499 (134.76 KB)

 Trainable params: 33,219 (129.76 KB)

 Non-trainable params: 1,280 (5.00 KB)

In [8]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=25
)


Epoch 1/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 23s 164ms/step - accuracy: 0.5087 - loss: 0.9856 - val_accuracy: 0.3814 - val_loss: 1.0981
Epoch 2/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7994 - loss: 0.5553 - val_accuracy: 0.3814 - val_loss: 1.0975
Epoch 3/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8716 - loss: 0.3941 - val_accuracy: 0.3814 - val_loss: 1.1094
Epoch 4/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9180 - loss: 0.2821 - val_accuracy: 0.3814 - val_loss: 1.0984
Epoch 5/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9330 - loss: 0.2230 - val_accuracy: 0.4359 - val_loss: 1.0356
Epoch 6/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9437 - loss: 0.1839 - val_accuracy: 0.6944 - val_loss: 0.8665
Epoch 7/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9459 - loss: 0.1759 - val_accuracy: 0.8056 - val_loss: 0.6251
Epoch 8/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9599 - loss: 0.1427 - val_accuracy: 0.8579 -

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model_float.tflite", "wb") as f:
    f.write(tflite_model)

print("Float TFLite model saved as model_float.tflite")


Saved artifact at '/tmp/tmp1le3u_l1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 49, 13, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  137367217873040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134288976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134289552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134288592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134289360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134289744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259612560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259613712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259612176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259611984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259613136:

In [10]:
def representative_dataset():
    for i in range(200):
        sample = X_train[i:i+1].astype(np.float32)
        yield [sample]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

int8_tflite_model = converter.convert()

with open("model_int8.tflite", "wb") as f:
    f.write(int8_tflite_model)

print("INT8 quantized TFLite model saved as model_int8.tflite")


Saved artifact at '/tmp/tmpv9gufcse'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 49, 13, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  137367217873040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134288976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134289552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134288592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134289360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367134289744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259612560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259613712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259612176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259611984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137367259613136:

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


INT8 quantized TFLite model saved as model_int8.tflite


In [11]:
import numpy as np
import tensorflow as tf

# Load model
interpreter = tf.lite.Interpreter(model_path="model_int8.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Run inference on 1 sample
sample = X_test[1:2]

# Convert to int8 scale
input_details = interpreter.get_input_details()[0]
scale, zero_point = input_details["quantization"]
sample_int8 = (sample / scale + zero_point).astype(np.int8)

interpreter.set_tensor(input_index, sample_int8)
interpreter.invoke()

output = interpreter.get_tensor(output_index)
print("TFLite prediction:", output)
print("Predicted class:", np.argmax(output))


TFLite prediction: [[ 118 -122 -125]]
Predicted class: 0


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [12]:
!cp model_int8.tflite kws_model.tflite
!xxd -i kws_model.tflite > kws_model.h
